In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

#python-louvain
#this module uses louvain method to detect communities in the network
import community.community_louvain as community_louvain

#STEP1
def get_APA7_names(namelist):
  names = []
  for x in namelist:
    Authors = x.split(";")
    name0 = []
    for y in Authors:
      Familyname = y.split(",")[0]
      if " " in Familyname:
        name0.append(Familyname.replace(" ", ""))
      else:
        name0.append(Familyname)
    names.append(name0)
  
  APA7_names = []
  for i in names:
    if len(i) > 2:
      APA7_names.append(i[0] + " et al. ")
    elif len(i) == 2:
      APA7_names.append(i[0] + " and " + i[1] + " ")
    else:
      APA7_names.append(i[0] + " ")
  #this will return a list
  return APA7_names

#STEP2, this combine STEP1 in. 
def gen_APA7style(Authors, Publication_Year):
  APA7style_years = Publication_Year.tolist()
  APA7style_names = get_APA7_names(Authors.tolist())
  APA7style = []
  for x, y in zip(APA7style_names, APA7style_years):
    citation = x + "(" + str(int(y)) + ")"
    APA7style.append(citation)
  
  #this will return a list
  return APA7style


#STEP3 For the References and References to edges
#generate a edge list for bipartite graph from "Cited References" in WoS excel output 


def get_References(Cited_References):
  References = [i.split("; ") for i in Cited_References] # changed in 1.02 , because I found that if I used ";" as a spliter, the first one will be different with others because of there is a " " at the start.
  for x in range(len(References)):
    References[x] = list(filter(lambda a: a != ' [No title captured]', References[x]))
  
  #This generate a list of References.
  return References


def gen_edges(Articles,  References_pdseries):
  References = get_References(References_pdseries)
  edges = []
  for Article, Reference_list in zip(Articles, References):
    for Reference in Reference_list:
      edges.append([Article, Reference])
  
  #This generate a list of edges (eg: [[A1,B1], [A2,B2]])
  return edges

#STEP4
#Nodes1 and Nodes2 and Edges for a whole Bipartite Graph
#need a list of Articles' name (eg: FirstName et al. (years))
def node0_node1_edges(Articles_lst, References_lst):
  edges = gen_edges(Articles_lst, References_lst)
  nodes_0 = [i[0] for i in edges]
  nodes_1 = list(set([i[1] for i in edges]))
  return {"node0": nodes_0, "node1": nodes_1, "edges": edges}

#STEP5
#generate a bipartite graph and only show nodes_0
def generate_bipartite_graph(nodes_0, nodes_1, edges):
  #generate a bipartite graph
  B = nx.Graph()
  B.add_nodes_from(nodes_0, bipartite=0)
  B.add_nodes_from(nodes_1, bipartite=1)
  B.add_edges_from(edges)
  #only show nodes_0
  G = nx.bipartite.weighted_projected_graph(B, nodes_0)
  #this return a nxGraph.object
  return G

#STEP6
#change a bipartite graph to a weighted graph

def weighted_graph(Graph, greater):
  nxweights = nx.get_edge_attributes(Graph, "weight")
  contain = dict((k, v) for k, v in nxweights.items() if v >= greater)

  #create node, edge , weighted_edge
  node_new = list(set([y for x in contain.keys() for y in x]))
  edge_new = list(contain.keys())
  edge_weights = list(contain.values())
  weighted_edge = [[edge[0], edge[1], w] for edge, w in zip(edge_new, edge_weights)]
  
  #generate the Graph
  G_new = nx.Graph()
  G_new.add_nodes_from(node_new)
  G_new.add_weighted_edges_from(weighted_edge)

  #this return a nxGraph.object
  return G_new

#STEP7
#clear the island which only have 2 connecting nodes
def clear_island(G):
  nodes_picker = [x for x in list(nx.connected_components(G)) if len(x) > 1]
  nodes_lst = [y for x in nodes_picker for y in x]
  edges_lst_old = list(G.edges)
  edges_lst_new = [(i[0], i[1]) for i in edges_lst_old if i[0] and i[1] in nodes_lst]
  attrs_old = nx.get_edge_attributes(G, "weight")
  attrs_new = {edges_lst_new[num]:{"weight":attrs_old[edges_lst_new[num]]} for num in range(len(edges_lst_new))}
  G_new = nx.Graph()
  G_new.add_nodes_from(nodes_lst)
  G_new.add_edges_from(edges_lst_new)
  nx.set_edge_attributes(G_new, attrs_new)
  return G_new

#STEP addition
#set nodes' attritutes
def set_nodes_attrs(G, df):
  col = df.columns.to_list()
  for i in col:
    nx.set_node_attributes(G, pd.Series(df[i], index=df.INDEX).to_dict(), i)

In [ ]:
#This is a function use all the functions above this to create a nxGraph from WoS excel
def generate_FinGraph(df_path, greator=100):
  #read and set the df
  df = pd.read_excel(df_path)
  needed_columns = ['Publication Type','Authors','Article Title', 'Publication Year', 'Source Title','Abstract','Cited References','Times Cited, WoS Core','DOI', 'Keywords Plus']

  #organize the df
  df = df[needed_columns]
  for i in needed_columns:
    if i != "Keywords Plus":
      df.dropna(subset=[i], inplace=True)
  
  df.reset_index(inplace=True)
  
  df["label"] = gen_APA7style(df["Authors"], df["Publication Year"])
  #set the INDEX
  df["INDEX"] = [int(i) for i in range(len(df))]
  
  
  #show how many samples we have
  print("The number of samples(N): ", len(df))

  #Nodes1 and Nodes2 and Edges for a whole Bipartite Graph
  N1N2E = node0_node1_edges(df["INDEX"], df["Cited References"]) # this is a dict
  
  #generate a bipartite graph object
  B_Graph = generate_bipartite_graph(N1N2E["node0"], N1N2E["node1"], N1N2E["edges"])#this is a bipartite graph object
  
  #set node_0's attributes
  set_nodes_attrs(B_Graph, df)

  #to get the top100 connecting strength
  weights = nx.get_edge_attributes(B_Graph, "weight")
  weights_lst = sorted(list(weights.values()), reverse=True)
  greater = weights_lst[greator]
  print("The connecting strength will be greater than: ", greater)
  
  #make the graph which we need
  Fin_Graph = weighted_graph(B_Graph, greater) #"greater" means connecting strength stronger than how many which need to input a int, and it's a ">="
  
  #set the Fin Graph's attrs
  df_B_Graph = pd.DataFrame.from_dict(dict(B_Graph.nodes(data=True)), orient='index')
  set_nodes_attrs(Fin_Graph, df)

  #Fin Graph to df (nodes)
  df_Fin_Graph = pd.DataFrame.from_dict(dict(Fin_Graph.nodes(data=True)), orient='index')
  
  #the Fin of Fin
  #clear the isolated nodes, which only connecting two nodes
  Fin_Fin_Graph = clear_island(Fin_Graph)

  #set the nodes' attributes
  set_nodes_attrs(Fin_Fin_Graph, df_Fin_Graph)

  #Betweens centralities
  bc = nx.betweenness_centrality(Fin_Fin_Graph)
  nx.set_node_attributes(Fin_Fin_Graph, bc, "betweenness")
  
  #weight
  Fin_weigth = nx.get_edge_attributes(Fin_Graph, "weight")
  Fin_weigth = {k:{"weight":v} for k, v in dict(Fin_weigth).items()}
  nx.set_edge_attributes(Fin_Fin_Graph, Fin_weigth)

  #set the communities in the nGraph
  partition = community_louvain.best_partition(Fin_Fin_Graph)
  nx.set_node_attributes(Fin_Fin_Graph, partition, name="Community")

  #print a len of nodes and edges
  print("Nodes data: " , len(list(Fin_Fin_Graph.nodes(data=True))))
  print("edges data: " , len(list(Fin_Fin_Graph.edges(data=True))))

  return Fin_Fin_Graph

In [ ]:
G_before2011 = generate_FinGraph("/content/BM_Aplus_Before2011.xlsx", greator=25)
print("\n----------------------------------------------------------\n")
G_2012to2015 = generate_FinGraph("/content/BM_Aplus_2012-2015.xlsx", greator=28)
print("\n----------------------------------------------------------\n")
G_2016to2019 = generate_FinGraph("/content/BM_Aplus_2016-2019.xlsx", greator=32)
print("\n----------------------------------------------------------\n")
G_2020to2021 = generate_FinGraph("/content/BM_Aplus_2020-2021.xlsx", greator=41)

The number of samples(N):  25
The connecting strength will be greater than:  2
Nodes data:  15
edges data:  36

----------------------------------------------------------

The number of samples(N):  28
The connecting strength will be greater than:  2
Nodes data:  22
edges data:  45

----------------------------------------------------------

The number of samples(N):  32
The connecting strength will be greater than:  5
Nodes data:  13
edges data:  31

----------------------------------------------------------

The number of samples(N):  41
The connecting strength will be greater than:  6
Nodes data:  25
edges data:  54


In [ ]:
df_lst = [pd.DataFrame.from_dict(dict(i.nodes(data=True)), orient='index') for i in [G_before2011, G_2012to2015, G_2016to2019, G_2020to2021]]

In [ ]:
for x, y in zip(df_lst, range(len(df_lst))):
  x.to_excel("BM_Aplus_" + str(y) + ".xlsx")

In [ ]:
edge_before2011 = nx.to_pandas_edgelist(G_before2011)
edge_2012to2015 = nx.to_pandas_edgelist(G_2012to2015)
edge_2016to2019 = nx.to_pandas_edgelist(G_2016to2019)
edge_2020to2021 = nx.to_pandas_edgelist(G_2020to2021)

edge_before2011.to_excel("edge_before2011.xlsx")
edge_2012to2015.to_excel("edge_2012to2015.xlsx")
edge_2016to2019.to_excel("edge_2016to2019.xlsx")
edge_2020to2021.to_excel("edge_2020to2021.xlsx")